# Punto 1

# Predicción de precios de casas

**Objetivo:** Utilizar el algoritmo de Regresión Lasso para la estimación de este modelo de predicción. Identifique cuáles son los atributos más relevantes para el problema.


**Conjunto de datos:** House Sales in King County, USA https://www.kaggle.com/harlfoxem/housesalesprediction

Para resolver el problema se empieza con análisis de datos se sigue con preparación de los mismos, entrenamiento del modelo y por último el análisis de resultados. Las desiciones tomadas han sido justificadas en las celdas en las que se realizan estas a través de anotaciones en el código. 

In [34]:
import pandas as pd 
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing

In [2]:
#Lectura de datos
data = pd.read_csv("kc_house_data.csv")

## Análisis de los datos

In [3]:
# Análisis inicial de los datos
data.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [4]:
#Información importante sobre las todos los datos disponibles
pandas_profiling.ProfileReport(data)

/Users/nicolasacevedo/anaconda3/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,21
Number of observations,21613
Total Missing (%),0.0%
Total size in memory,3.5 MiB
Average record size in memory,168.0 B
Numeric,19
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [5]:
#Ver valores ausentes 
data.isna().sum()
#No hay valores nulos 

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

### Conclusiones análisis de datos 

De acuerdo con los resultados obtenidos se pueden identificar con facilidad las variables continuas y categoricas que se obtienen. Se observa que exiten variables categoricas binarias que no serán modificadas debido a que ya se encuentran de forma que el algorítmo pueda trabajar. Por otro lado el "zip code", es una variable categórica que es necesario modificar y más adelante se justificará la decisión tomada sobre esta variable. Sobre las variables correlacionadas se toma la decisión de no hacer modificaciones debido a que a través de los métodos de regularización se puede controlar la complejidad e incluir las variables significativas y no tener en cuenta las variables que tienen una alta correlación si ya ha sido incluida esta información en las otras variables. Por otro lado, se puede ver que no hay valores nulos por lo que no se realizan modificaciones para resolver este problema. 

## Preparación de datos

In [6]:
#Eliminar las columnas que no sirven como ID porque no tienen información importante para la predicción
del data["id"]

In [7]:
#Crear una columna con los años en los que se realizó la venta
#Se tiene una fecha y se obtiene el año de esta debido a que se va utilizar para calcular la edad de la casa cuando se vende
data["yearSold"] = data["date"].str[0:4]
del data["date"]
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yearSold
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015


In [8]:
#Creo una nueva columna que el año en que se le realizó una modificación a la casa (Sea que se haya contruido o se haya remodelado)
#Se hace esta modificación para incluuir el efecto de las remodelaciones en el precio de una casa
data ["yearModified"] = data[["yr_built", "yr_renovated"]].apply(max, axis = 1)
del data["yr_built"]
del data["yr_renovated"]
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,yearSold,yearModified
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,98178,47.5112,-122.257,1340,5650,2014,1955
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,98125,47.7210,-122.319,1690,7639,2014,1991
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,98028,47.7379,-122.233,2720,8062,2015,1933
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,98136,47.5208,-122.393,1360,5000,2014,1965
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,98074,47.6168,-122.045,1800,7503,2015,1987


In [9]:
#Crear una columna con la diferencia entre los años de venta y el año en que se construyó/remodelo (Vejez de la casa hasta cuando se vendió)
data["yearSold"] = pd.to_numeric(data["yearSold"])
data["yearsOld"] = data["yearSold"] - data["yearModified"]

In [10]:
#Elimino las columnas que no utilizaré, porque ya ha sido incluida esta información en otras columnas
del data["yearSold"]
del data["yearModified"]

In [11]:
#Revisón de los datos con las modificaciones
data.head(n=100)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,yearsOld
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,98178,47.5112,-122.257,1340,5650,59
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,98125,47.7210,-122.319,1690,7639,23
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,98028,47.7379,-122.233,2720,8062,82
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,98136,47.5208,-122.393,1360,5000,49
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,98074,47.6168,-122.045,1800,7503,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,905000.0,4,2.50,3300,10250,1.0,0,0,3,7,2390,910,98040,47.5873,-122.249,1950,6045,23
96,247500.0,3,1.75,1960,15681,1.0,0,0,3,7,1960,0,98032,47.3576,-122.277,1750,15616,48
97,199000.0,4,1.50,1160,6400,1.0,0,0,4,7,1160,0,98023,47.3036,-122.378,1160,6400,39
98,314000.0,3,1.75,1810,41800,1.0,0,0,5,7,1210,600,98038,47.4109,-121.958,1650,135036,34


In [12]:
#Se considera que la longitud de la casa no son variables iimportantes para la predicción ya que no hya una relación de orden en las variables y no se puede considerar una mayor latitud/longituud como un mayor precio y biceversa
del data["lat"]
del data["long"]


In [13]:
#Se observa el número de datos por cada zip code y se puede ver que hay suficientes datos para considerarlo como una variables categórica y por lo tanto se realiza el manejo como dummies para poder incluir esta variable en el modelo predictivo
dataD = pd.get_dummies(data, columns=["zipcode"])
#Se verifica que haya suficientes datos de todas las categorias (Hay por lo menos 50 datos en las categorias)
dataD.columns.values[dataD.sum() < 50]

array([], dtype=object)

In [15]:
# Variable que se va a predecir 
y = dataD["price"]
del dataD["price"]

In [16]:
dataD.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,...,0,0,0,0,0,0,1,0,0,0
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,...,0,0,0,0,0,0,0,0,0,0
2,2,1.00,770,10000,1.0,0,0,3,6,770,...,0,0,0,0,0,0,0,0,0,0
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,...,0,0,0,0,0,0,0,0,0,0
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = dataD

In [18]:
X.head(n = 10)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,...,0,0,0,0,0,0,1,0,0,0
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,...,0,0,0,0,0,0,0,0,0,0
2,2,1.00,770,10000,1.0,0,0,3,6,770,...,0,0,0,0,0,0,0,0,0,0
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,...,0,0,0,0,0,0,0,0,0,0
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,...,0,0,0,0,0,0,0,0,0,0
5,4,4.50,5420,101930,1.0,0,0,3,11,3890,...,0,0,0,0,0,0,0,0,0,0
6,3,2.25,1715,6819,2.0,0,0,3,7,1715,...,0,0,0,0,0,0,0,0,0,0
7,3,1.50,1060,9711,1.0,0,0,3,7,1060,...,0,0,0,0,0,0,0,0,1,0
8,3,1.00,1780,7470,1.0,0,0,3,7,1050,...,1,0,0,0,0,0,0,0,0,0
9,3,2.50,1890,6560,2.0,0,0,3,7,1890,...,0,0,0,0,0,0,0,0,0,0


In [42]:
X.apply(pd.to_numeric)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,...,0,0,0,0,0,0,1,0,0,0
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,...,0,0,0,0,0,0,0,0,0,0
2,2,1.00,770,10000,1.0,0,0,3,6,770,...,0,0,0,0,0,0,0,0,0,0
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,...,0,0,0,0,0,0,0,0,0,0
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,3,2.50,1530,1131,3.0,0,0,3,8,1530,...,0,0,0,0,0,0,0,0,0,0
21609,4,2.50,2310,5813,2.0,0,0,3,8,2310,...,1,0,0,0,0,0,0,0,0,0
21610,2,0.75,1020,1350,2.0,0,0,3,7,1020,...,0,0,0,0,0,0,0,0,0,0
21611,3,2.50,1600,2388,2.0,0,0,3,8,1600,...,0,0,0,0,0,0,0,0,0,0


In [43]:
#División de los datos en train y test para poder revisar el modelo
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [55]:
X_train_s

array([[-2.50643639, -1.45004618, -1.21254876, ..., -0.07596488,
        -0.11289826, -0.12311076],
       [-0.39040069, -0.47752886, -0.61772898, ..., -0.07596488,
        -0.11289826, -0.12311076],
       [-0.39040069,  0.17081602, -0.4446905 , ..., -0.07596488,
        -0.11289826, -0.12311076],
       ...,
       [-0.39040069,  0.17081602,  0.30153796, ..., -0.07596488,
        -0.11289826, -0.12311076],
       [ 0.66761716, -0.15335642,  0.31235286, ..., -0.07596488,
        -0.11289826, -0.12311076],
       [ 0.66761716,  0.17081602,  0.32316777, ..., -0.07596488,
        -0.11289826, -0.12311076]])

In [52]:
#Se estandarizan los datos, teniendo en cuenta la media y la varianza de los datos de train. 
s_scaler = preprocessing.StandardScaler()
X_train_s = s_scaler.fit_transform(X_train)
X_test_s = s_scaler.transform(X_test)

## Entrenamiento del modelo (LASSO)

In [61]:
#Se hace la regresión Lasso
#10 k-fold para hacer la validación a través de CrossValidation
#No se utiiliza un arreglo de alphas porque automáticamente la librería utiliza unos por defecto. 
reg = LassoCV(cv=10, random_state=0).fit(X_train_s, y_train)

## Análisis de resultados

In [62]:
#Se valida el resultado del modelo de predicción utilizando el R^2 
#Se obtiene un resultado bueno de predicción
reg.score(X_test_s,y_test)

0.8082044036352494

In [63]:
#Se obtiene los coefcientes de la regresión
coef = abs(reg.coef_)

In [64]:
#Feature selection 
#Se obtiene las variables más significativas para el modelo de predicción, los betas asociados más grandes para el modelo Lasso.
#Es importante considerar que los betas pueden no considerar algunas variables que en un inicio parecen importantes porque primero incluye las variables más significativas y si hay una muy correlacionada con estas no se va incluir porque no aporta nueva información. 
model = SelectFromModel(reg, threshold="mean", prefit = True)
new = model.get_support()

In [65]:
#Nombres de las variables significativas. 
X_train.columns[new]

Index(['bedrooms', 'sqft_living', 'floors', 'waterfront', 'view', 'grade',
       'sqft_above', 'zipcode_98001', 'zipcode_98003', 'zipcode_98004',
       'zipcode_98023', 'zipcode_98033', 'zipcode_98038', 'zipcode_98039',
       'zipcode_98040', 'zipcode_98042', 'zipcode_98058', 'zipcode_98092',
       'zipcode_98102', 'zipcode_98103', 'zipcode_98105', 'zipcode_98107',
       'zipcode_98109', 'zipcode_98112', 'zipcode_98115', 'zipcode_98117',
       'zipcode_98119', 'zipcode_98122', 'zipcode_98199'],
      dtype='object')

#### Se realiza el análisis sin incluir las variables dummies ("zipcode") pero se obtienen resultados inferiores en cuanto a la precisión sobre los datos de test, por lo tanto se decide dejarlas en el modelo debido a que predice mejor. 

### Conclusiones análisis de resultados

Se puede observar que a través de los datos que se obtienen se puede llegar a predecir el precio de la casa con algunas de las variabels incluidas. En primer lugar, en cuanto a la predicción del modelo. Se puede observar que el modelo en general es bueno y predice adecuadamente los precios de la casa, cuando se evalúa con el conjunto de test. Se obtiene un valor de 0.8 para el R^2, lo que quiere decir que el modelo se adecua bastante bien a la función objetivo que se busca. Por otro lado, es importante mencionar que el modelo fue entrenado con varias modificaciones de los datos y la que se presenta en el informe es la mejor que se pudo encontrar. Se evaluó el modelo, sin utilizar las variables dummies ("zipcode") y el valor de R^2 baja a un valor de 0.69. En cuanto a las variables más significativas para la predicción de acuerdo con el modelo obtenido, se puede ver que se encuentran agrupadas en unas especificas: el número de cuartos, los pies cuadrados de la casa, el número de pisos, si tiene vista al mar, si tiene vista, la calificación que se le ha dado a la casa, los pies cudrados de la terraza y el lugar(el cual se puede identificar por el zipcode)  Estas se pueden encontrar al observar los coeficiente que acompañan a la regresión LASSO (betas de la regresión). Los datos de las variables significativas van de acuerdo a lo que se espera de la realidad y por lo tanto, se vuelven significativas si se busca tener una casa que tenga un precio alto, enofcarse en tener una casa con las variables mencionadas. 